##  Decorators

In [1]:
def decorate1(func):
    def inner():
        print("apply decorate1")
        r = func()
        return r

    return inner


@decorate1
def hello():
    """doc di hello"""
    print("hello, world!")


@decorate1
def bye():
    print("bye bye")


hello()
bye()

apply decorate1
hello, world!
apply decorate1
bye bye


In [2]:
help(hello)   # mi da info sulla INNER! con @decorate
#con help va bene ma con "??" no(?)

Help on function inner in module __main__:

inner()



In [2]:
def decorate1(func):
    def inner():
        print("decorate 1")
        r = func()
        # ...
        return r

    return inner


def hello():
    print("hello, world!")
    return "pippo"


hello = decorate1(hello)  # what @decorate1 does !

a = hello()  # executes inner
print(a)

decorate 1
hello, world!
pippo


In [4]:
help(hello)    #mi dà info su hello ok

Help on function inner in module __main__:

inner()



In [5]:
def decorate2(func):
    def inner():
        print("apply decorate2")
        r = func()
        return r

    return inner


@decorate1
@decorate2
def hello12():
    print("hello, world!")


hello12()  # same as hello12 = decorate1(decorate2(hello12))

decorate 1
apply decorate2
hello, world!


In [6]:
@decorate2
@decorate1
def hello21():
    print("hello, world!")


hello21()  # same as hello21 = decorate2(decorate1(hello21))

apply decorate2
decorate 1
hello, world!


### Remember to use `@functools.wraps`

In [7]:
import functools


def decorate1(func):
    @functools.wraps(func)  #SERVE PER FAR COMPARIRE LA DOC DELLA FUNZIONE INTERNA
    def inner():
        print("apply decorate1")
        func()

    return inner


@decorate1
def hello():
    "doc for hello"
    print("hello, world!")


help(hello)

Help on function hello in module __main__:

hello()
    doc for hello



In [8]:
hello??

Signature: hello()
Source:   
@decorate1
def hello():
    "doc for hello"
    print("hello, world!")
File:      /tmp/ipykernel_17003/135412615.py
Type:      function


In [9]:
hello.__doc__ = "pippo"

In [10]:
help(hello)

Help on function hello in module __main__:

hello()
    pippo



##


###  How to pass arguments to the inner function

In [11]:
# adapted from Fluent Python
import functools


def args_to_string(*args, **kw):
    arg_str = ()
    if args:
        arg_str += ((",".join(str(arg) for arg in args)),)
    if kw:
        arg_str += ((", ".join(("{0}={1}".format(k, v) for k, v in kw.items()))),)
    return ",".join(a for a in arg_str)

In [12]:
from time import perf_counter, sleep


def time_this(func):
    @functools.wraps(func)
    def decorated(*args, **kw):
        t0 = perf_counter()
        result = func(*args, **kw) 
        # COSI POSSO APPLICARE TIMETHIS A FUNZIONI CHE PRENDANO QUALSIASI NUMERO DI ARGS
        t1 = perf_counter()
        name = func.__name__
        arg_str = args_to_string(*args, **kw)
        # print('{}({}): [{:0.8f}]'.format(name, arg_str,t1-t0))
        # print('%s(%s): [%0.8f s]' % (name, arg_str, t1-t0))
        print(f"{name}({arg_str}):[{t1-t0:0.8f} s]")    #up to 8 digits
        return result

    return decorated


@time_this
def wait(seconds):
    sleep(seconds)

#LRU CACHE: construct a dict in which the key is n and the value is the result 


@functools.lru_cache()  #parametr decor, ha (). Posso dirgli quanti risult ricordare, default 128
@time_this
def factorial(n):
    return 1 if n < 2 else n * factorial(n - 1)


@time_this
def sum(a, b):
    return a + b


@time_this
def dummy(*args, **kw):
    a = args
    b = kw


wait(0.3)
factorial(10)
sum(4, 5)
dummy("pos", "second", a="a", b="b")

wait(0.3):[0.30145539 s]
factorial(1):[0.00000035 s]
factorial(2):[0.00005955 s]
factorial(3):[0.00010783 s]
factorial(4):[0.00015481 s]
factorial(5):[0.00020057 s]
factorial(6):[0.00024819 s]
factorial(7):[0.00029550 s]
factorial(8):[0.00034096 s]
factorial(9):[0.00038670 s]
factorial(10):[0.00046354 s]
sum(4,5):[0.00000032 s]
dummy(pos,second,a=a, b=b):[0.00000081 s]


In [13]:
factorial(21)

factorial(11):[0.00000048 s]
factorial(12):[0.00202147 s]
factorial(13):[0.00212162 s]
factorial(14):[0.00218587 s]
factorial(15):[0.00223854 s]
factorial(16):[0.00231387 s]
factorial(17):[0.00250859 s]
factorial(18):[0.00258185 s]
factorial(19):[0.00265482 s]
factorial(20):[0.00271186 s]
factorial(21):[0.00277447 s]


51090942171709440000

In [14]:


def parametrized_time_this(check=True): #questa è una funzione, riturna il decorator
    def decorator(func):
        if not check:
            return func

        @functools.wraps(func) 
        def decorated(*args, **kw):    #questa dà la funzione decorated
            t0 = perf_counter()
            result = func(*args, **kw)
            t1 = perf_counter()
            name = func.__name__
            arg_str = args_to_string(*args, **kw)
            print(f"{name}({arg_str}):[{t1-t0:0.8f} s]")
            return result

        return decorated

    return decorator  # <-- returns the actual decorator


debug = True
#posso filtrare quali funzioni eseguire cambiando una riga

@parametrized_time_this(debug)
def wait(seconds):
    print("going to sleep for", seconds, "seconds")
    sleep(seconds)
    print("woke up!")


wait(0.4)

going to sleep for 0.4 seconds
woke up!
wait(0.4):[0.40147535 s]


### Decorators as function objects

In [15]:
#function objects are something that accept a call

class TimeThis:
    def __init__(self, func):  # <--   defin un init che prende una func
        self._func = func  #  <--  gli elem che iniziano con "_" sono privati
        functools.update_wrapper(self, func)  # <-- come wraps credo, serve per mantenere la doc della funzione wrapped

    def __call__(self, *args, **kw):
        t0 = perf_counter()
        result = self._func(*args, **kw)     # <--
        t1 = perf_counter()
        name = self._func.__name__  # <-- aggiunto SELF davanti
        arg_str = args_to_string(*args, **kw)
        print(f"{name}({arg_str}):[{t1-t0:0.8f} s]")

        return result

    
# ni: riascoltare qui

@TimeThis
def wait(seconds):
    "doc"
    print("going to sleep for", seconds, "seconds")
    sleep(seconds)
    print("woke up!")


wait(0.4)

going to sleep for 0.4 seconds
woke up!
wait(0.4):[0.40199842 s]


In [16]:
#functools.update_wrapper??

wait??

Signature:      wait(seconds)
Call signature: wait(*args, **kw)
Type:           TimeThis
String form:    <__main__.TimeThis object at 0x7fbe1c3aa580>
File:           /tmp/ipykernel_17003/2980550374.py
Source:        
@TimeThis
def wait(seconds):
    "doc"
    print("going to sleep for", seconds, "seconds")
    sleep(seconds)
    print("woke up!")


## parametrized decor by class

In [17]:
class ParametrizedTimeThis:
    def __init__(self, check=True):
        self.check = check

    def __call__(self, func):
        if self.check:
            # return TimeThis(func)

            @functools.wraps(func)
            @TimeThis
            def wrapper(*args, **kwargs):
                return func(*args, **kwargs)

            return wrapper
        return func


@ParametrizedTimeThis(True)
def wait(seconds):
    print("going to sleep for", seconds, "seconds")
    sleep(seconds)
    print("woke up!")


wait(0.4)

going to sleep for 0.4 seconds
woke up!
wrapper(0.4):[0.40268133 s]


In [18]:
wait??

Signature:      wait(seconds)
Call signature: wait(*args, **kw)
Type:           TimeThis
String form:    <__main__.TimeThis object at 0x7fbe1c3aa220>
File:           /tmp/ipykernel_17003/3138198677.py
Source:        
@ParametrizedTimeThis(True)
def wait(seconds):
    print("going to sleep for", seconds, "seconds")
    sleep(seconds)
    print("woke up!")


In [19]:
PTT = ParametrizedTimeThis(True)


@PTT
def dummy(*args, **kw):
    pass


dummy(0.4)

wrapper(0.4):[0.00000066 s]


In [20]:
dummy??

Signature:   dummy(*args, **kw)
Type:        TimeThis
String form: <__main__.TimeThis object at 0x7fbe1c3ab520>
File:        /tmp/ipykernel_17003/1511016295.py
Source:     
@PTT
def dummy(*args, **kw):
    pass
